In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style ('dark')


In [ ]:
df = pd.read_csv("/kaggle/input/students-performance-in-exams/StudentsPerformance.csv", sep = ",")
df.head()

In [ ]:
df.isnull().sum()

# Finding Unique Values in each column

In [ ]:
for col in df.columns.unique():
    print('\n', col ,'\n', df[col].unique())

In [ ]:
df.info()

# Histogram of reading, writing & math score

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(18, 10))
#fig, axs = plt.subplots(ncols=4,figsize=(20,5))
sns.histplot(x=df['math score'], ax = axes[0,0], color = 'b')
sns.histplot(x=df['reading score'], ax = axes[0,1], color = 'g')
sns.histplot(x=df['writing score'], ax = axes[1,0], color="r")
#sns.countplot(df['lunch'], ax = axs[0,3], palette="deep")


# Count plot of gender, race, level of eductaion and test preparation

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(18, 10))
#fig, axs = plt.subplots(ncols=4,figsize=(20,5))
sns.countplot(x=df['gender'], ax = axes[0,0])
sns.countplot(x=df['race/ethnicity'], ax = axes[0,1],palette="vlag")
sns.countplot(x=df['parental level of education'], ax = axes[1,0],  palette="rocket")
sns.countplot(x=df['test preparation course'], ax = axes[1,1], palette="deep")



# How reading & math score are influnced by different parameters

In [ ]:
plt.figure(figsize = (10,7))
fig, axes = plt.subplots(2, 2, figsize=(18, 10))
sns.scatterplot(x= 'math score', y = 'reading score' , hue = 'parental level of education', data = df, ax=axes[0, 0], palette="deep")
sns.scatterplot(x= 'math score', y = 'reading score' , hue = 'race/ethnicity', data = df, ax=axes[0, 1])
sns.scatterplot(x= 'math score', y = 'reading score' , hue = 'lunch', data = df, ax=axes[1, 0])
sns.scatterplot(x= 'math score', y = 'reading score' , hue = 'test preparation course', data = df, ax=axes[1, 1])



In [ ]:
plt.figure(figsize=(20,15))
sns.pairplot(df, hue = 'parental level of education')
plt.show()

In [ ]:
df.columns

# How does Each Level of Parental Eductaion affects the reading & Math scores

In [ ]:
plt.figure(figsize=(10,7))
g = sns.FacetGrid(df, col="race/ethnicity", hue ='parental level of education' )
g.map(sns.scatterplot, "math score", "reading score", alpha=.7)
g.add_legend()

# Which race & gender are doing good in math & reading

In [ ]:
g = sns.FacetGrid(df, col="race/ethnicity", hue = 'gender')
g.map(sns.scatterplot, "math score", "reading score", alpha=.7)
plt.figure(figsize=(15,7))
g.add_legend()

In [ ]:
df1 = df.copy()
df1.head(2)

# Converting Categorical features

In [ ]:
df1 = pd.get_dummies(df1)
df1.head()

In [ ]:
df1.shape

# Establishing corleation with various parameters

In [ ]:
plt.figure(figsize=(10,7))
corr = df1.corr()
sns.heatmap(corr, linewidth = 0.5, cmap = 'coolwarm')

# Predicting Math Score

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
rf = RandomForestRegressor()
dt = DecisionTreeRegressor()


In [ ]:
X= df1.drop(['math score'], axis = 1)
y = df1['math score']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 43, test_size = 0.2)

In [ ]:
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)


In [ ]:
#@from sklearn.metrics import r2_score
#from sklearn.metrics import mean_squared_error

#from sklearn.metrics import accuracy_score
#accu_score_rf = accuracy_score(y_test, y_pred_rf)

#print('accu_score_rf:', accu_score_rf)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
df2 = df.copy()
#df2 = le.fit_transform(df2)
col = ['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course']
for col in col:
    df2[col] = le.fit_transform(df2[col])
df2.head()

In [ ]:
df2.info()

In [ ]:
X= df2.drop(['math score'], axis = 1)
y = df2['math score']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 43, test_size = 0.2)

In [ ]:
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test).round(0)

In [ ]:
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

from sklearn.metrics import accuracy_score
r2_score_rf = metrics.r2_score(y_test, y_pred_rf)
mse_rf = metrics.mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)


print('r2_score_rf:', r2_score_rf, '\n', 'MSE:', mse_rf, '\n', 'RMSE: ', rmse_rf)

In [ ]:
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test).round(0)

r2_score_dt = metrics.r2_score(y_test, y_pred_dt)
mse_dt = metrics.mean_squared_error(y_test, y_pred_dt)
rmse_dt = np.sqrt(mse_dt)

print('r2_score_dt:', r2_score_dt, '\n', 'MSE:', mse_dt, '\n', 'RMSE: ', rmse_dt)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X, y)

In [ ]:
rf_random.best_params_

In [ ]:
!pip install xlrd
!pip install AutoViz

In [ ]:
from autoviz.AutoViz_Class import AutoViz_Class
#Instantiate the AutoViz class
AV = AutoViz_Class()

In [ ]:
df = AV.AutoViz('/kaggle/input/students-performance-in-exams/StudentsPerformance.csv', depVar = 'parental level of education')